In [26]:
import pandas as pd
import numpy as np

import json


import requests

from pandas import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

Reading the data scrapped from the Wikipedia page into a Dataframe

In [3]:
df=pd.read_csv(r'/resources/data/TorontoData.csv')
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queens Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Cleaning the Dataframe of Boroughs with Not Assigned Values


In [4]:
for column in df.columns:
   df = df[df['Borough']!='Not assigned']
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queens Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill ,Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Getting the shape of the Dataframe

In [5]:
df.shape

(103, 3)

We now associate the locations to their respective latitudes and longitudes. I'm uploading a .csv file of the same.

In [6]:
geo_df=pd.read_csv(r'/resources/data/GeoToronto.csv')
geo_df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [7]:
!pip install geocoder

import geocoder # import geocoder


In [1]:
!pip install geopy --user
import  os, geopy

%pip search geopy
from geopy.geocoders import Nominatim

tornado-geopy (0.1.0)         - tornado-geopy is an asynchronous version of
                                the awesome geopy library.
geopy (2.0.0)                 - Python Geocoding Toolbox
  INSTALLED: 2.0.0 (latest)
swisslandstats-geopy (0.8.0)  - Python for the land statistics datasets from
                                the SFSO
pivotal-geopy (1.0.0)         - Python Geocoding Toolbox
Note: you may need to restart the kernel to use updated packages.


We now plot the above locations on a map of Toronto

In [8]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(geo_df['Latitude'], geo_df['Longitude'], geo_df['Borough'], geo_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
For simplicity, we only take explore those boroughs which have Toronto in them.

In [20]:
tor_data = geo_df[geo_df['Borough'].str.contains("Toronto")]
tor_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth, West Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Now we explore the city of Toronto with Foursqaure API.

In [18]:
CLIENT_ID = 'GGNHWFPUG4BQYY3RP5PVUIPAUXTI32IBP3V4VBUN054HGKJ0' 
CLIENT_SECRET = 'N1KFLSJIUL0WNO53X0MTVCO4PJ4X5JZ0ISVGWHLLSQECSRLH' 
VERSION = '20180605' 
LIMIT = 39 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GGNHWFPUG4BQYY3RP5PVUIPAUXTI32IBP3V4VBUN054HGKJ0
CLIENT_SECRET:N1KFLSJIUL0WNO53X0MTVCO4PJ4X5JZ0ISVGWHLLSQECSRLH


In [28]:
neighborhood_latitude = tor_data.loc[41, 'Latitude']
neighborhood_longitude = tor_data.loc[41, 'Longitude'] 

neighborhood_name = tor_data.loc[41, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Danforth, West Riverdale are 43.6795571, -79.352188.


In [31]:
LIMIT = 39
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=GGNHWFPUG4BQYY3RP5PVUIPAUXTI32IBP3V4VBUN054HGKJ0&client_secret=N1KFLSJIUL0WNO53X0MTVCO4PJ4X5JZ0ISVGWHLLSQECSRLH&ll=43.6534817,-79.3839347&v=20180605&radius=500&limit=39'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa86a2adc3fa014e676660f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [34]:
venues = results['response']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,headerLocation,headerFullLocation,headerLocationGranularity,totalResults,groups,suggestedFilters.header,suggestedFilters.filters,suggestedBounds.ne.lat,suggestedBounds.ne.lng,suggestedBounds.sw.lat,suggestedBounds.sw.lng
0,Bay Street Corridor,"Bay Street Corridor, Toronto",neighborhood,90,"[{'type': 'Recommended Places', 'name': 'recom...",Tap to show:,"[{'name': 'Open now', 'key': 'openNow'}]",43.657982,-79.377727,43.648982,-79.390143
